In [39]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [40]:
baseUrl = "https://www.24ur.com"
articleArchiveSoup = BeautifulSoup(urlopen(baseUrl + "/arhiv"), "html.parser")

In [51]:
import os
from selenium import webdriver

# to prevent selenium browser from opening during running
os.environ['MOZ_HEADLESS'] = '1'

browser = webdriver.Firefox(executable_path='/home/majsavli/geckodriver')

In [79]:
# how many article pages to scrap
PAGE_LIMIT = 20
parsedComments = []

for page in range(PAGE_LIMIT):
    print("Page: ", page + 1)
    articles = articleArchiveSoup.find_all("a", {"class": "timeline__item"})

    for article in articles:
        articleUrl = baseUrl + article.attrs["href"]
        print(articleUrl)
        browser.get(articleUrl)
        comments = browser.find_elements_by_class_name('comment')

        for comment in comments:
            if len(comment.get_attribute('class').split(' ')) == 1:
                commentText = comment.find_element_by_class_name('comment__body')
                try:
                    rating = comment.find_element_by_class_name('comment__rating')
                    parsedComments.append((commentText.text, rating.text))
                except:
                    parsedComments.append((commentText.text, "-"))

Page:  1
https://www.24ur.com/sport/nogomet/serie-a-ilicic-po-slovesu-od-lige-prvakov-v-veroni-hellas-atalanta-juve-benevento-inzaghi-ronaldo-fiorentina-milan-roma-napoli.html
https://www.24ur.com/sport/zimski_sporti/schwarz-pricakovano-v-vodstvu-hadalin-z-visokim-zaostankom-19.html
https://www.24ur.com/sport/zimski_sporti/kriznarjeva-po-izgubljeni-majici-vodilne-spet-v-lov-za-takanasijevo-v-niznjem-tagilu.html
https://www.24ur.com/novice/crna-kronika/v-mariboru-zgorela-tovorna-vozila-v-ograjenem-prostoru-vzrok-neznan.html
https://www.24ur.com/popin/glasba/karmen-in-september-predstavljajo-videospot-za-pesem-dobro-jutro-dobri-ljudje.html
https://www.24ur.com/novice/tujina/policija-v-nemciji-delezna-kritik-zaradi-nasilnega-ukrepanja-na-protestih.html
https://www.24ur.com/novice/tujina/kathleen-folbigg.html
https://www.24ur.com/popin/tuja-scena/pevka-bjork-navdusena-nad-izbruhom-vulkana-v-blizini-njenega-doma.html
https://www.24ur.com/novice/tujina/eu-poziva-turcijo-zenske-si-zasluzijo-m

In [80]:
import re

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


filename = "../24ur_comments.csv"
f = open(filename, "w")

headers = "comment, like_ratio\n"
f.write(headers)

for text, likes in parsedComments:
    f.write("'" + remove_emoji(text) + "'" + "," + likes + "\n")

f.close()